<h2>Rubert на упоминаниях RDRS, которые содержат общие с корпусом CADEC PT коды</h2>

In [1]:
transformer_model_path = 'DeepPavlov/rubert-base-cased'#'sagteam/xlm-roberta-large-sag'
USE_CUDA = True
ADD_CONTEXT = False
USE_CYCLR = False

In [2]:
import pandas as pd
from vectorization import ConceptVectorizer
import torch
from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold

In [3]:
#альтернатива
CV = ConceptVectorizer(transformer_model_path, '../../Data/External/pt_rus.asc', \
                             use_concept_less=False, use_model=False, use_cuda=USE_CUDA)
CV_with_conceptless = ConceptVectorizer(transformer_model_path, '../../Data/External/pt_rus.asc', \
                             use_concept_less=True, use_model=False, use_cuda=USE_CUDA)

In [4]:
CV.thesaurus_embeddings = torch.load('./Model_weights/rubert_thesaurus_embeddings_meddra_origin.pt')
CV.normalization_mode = 'mean_pooling'

In [5]:
n_splits=5
kf = KFold(n_splits=n_splits, random_state=42, shuffle=True)

In [6]:
df = pd.read_csv('../../Data/RDRS_intersection_part.csv')


all_splits = dict()

df['pt_id'] = df['pt_id'].apply(lambda x: str(x))

terms = df['phrase'].to_numpy()
concepts = df['pt_id'].to_numpy()

split_idx = 1
for train_index, test_index in kf.split(df['phrase']):
    all_splits[split_idx]=dict()
    all_splits[split_idx]['train'] = dict()
    all_splits[split_idx]['test'] = dict()
    all_splits[split_idx]['validation'] = dict()
    all_splits[split_idx]['test']['phrases'] = list(terms[test_index])
    all_splits[split_idx]['test']['concepts'] = list(concepts[test_index])
    
    train_index, val_index = train_test_split(train_index, test_size=0.1, random_state=42)
    
    all_splits[split_idx]['train']['phrases'] = list(terms[train_index])
    all_splits[split_idx]['train']['concepts'] = list(concepts[train_index])
    all_splits[split_idx]['validation']['phrases'] = list(terms[val_index])
    all_splits[split_idx]['validation']['concepts'] = list(concepts[val_index])
    
    
    print('Сплит %s'%split_idx)
    print('Всего фраз в трейне: %s'%len(all_splits[split_idx]['train']['phrases']))
    print('Всего фраз в тесте: %s'%len(all_splits[split_idx]['test']['phrases']))
    print('Всего фраз в девелопе: %s'%len(all_splits[split_idx]['validation']['phrases']))

    print('Уникальных фраз в трейне: %s'%len(set(all_splits[split_idx]['train']['phrases'])))
    print('Уникальных фраз в тесте: %s'%len(set(all_splits[split_idx]['test']['phrases'])))
    print('Уникальных фраз в девелопе: %s'%len(set(all_splits[split_idx]['validation']['phrases'])))
    
    print('%s концептов не входящих либо в трейн, либо в тест'%len(set.union(set(all_splits[split_idx]['train']['concepts']), set(all_splits[split_idx]['test']['concepts'])) - set.intersection(set(all_splits[split_idx]['test']['concepts']), set(all_splits[split_idx]['train']['concepts']))))
    print('%s концептов, которые есть в тесте, но нет в трейне'%len(set(all_splits[split_idx]['test']['concepts']) - set(all_splits[split_idx]['train']['concepts'])))
    print('%s концептов, которые есть в трейне, но нет в тесте'%len(set(all_splits[split_idx]['train']['concepts']) - set(all_splits[split_idx]['test']['concepts'])))
    print('\n')
    split_idx+=1

Сплит 1
Всего фраз в трейне: 5665
Всего фраз в тесте: 1574
Всего фраз в девелопе: 630
Уникальных фраз в трейне: 2878
Уникальных фраз в тесте: 1011
Уникальных фраз в девелопе: 467
83 концептов не входящих либо в трейн, либо в тест
11 концептов, которые есть в тесте, но нет в трейне
72 концептов, которые есть в трейне, но нет в тесте


Сплит 2
Всего фраз в трейне: 5665
Всего фраз в тесте: 1574
Всего фраз в девелопе: 630
Уникальных фраз в трейне: 2891
Уникальных фраз в тесте: 1009
Уникальных фраз в девелопе: 442
89 концептов не входящих либо в трейн, либо в тест
7 концептов, которые есть в тесте, но нет в трейне
82 концептов, которые есть в трейне, но нет в тесте


Сплит 3
Всего фраз в трейне: 5665
Всего фраз в тесте: 1574
Всего фраз в девелопе: 630
Уникальных фраз в трейне: 2901
Уникальных фраз в тесте: 984
Уникальных фраз в девелопе: 476
83 концептов не входящих либо в трейн, либо в тест
6 концептов, которые есть в тесте, но нет в трейне
77 концептов, которые есть в трейне, но нет в тес

In [8]:
from tqdm import trange
from tqdm import tqdm
from sklearn.metrics import classification_report, f1_score
from evaluator import Evaluator
import numpy as np
import torch.optim as optim
import torch.nn as nn
import torch
import numpy as np
from tools.pytorchtools import EarlyStopping
import pandas as pd
from torch.optim.lr_scheduler import CyclicLR
from dataset import *
from models import CADEC_SoTa

#Гиперпараметры
batch_size=16
epochs = 50

all_splits_score_dfs = []

input_format = 'phrases' if not ADD_CONTEXT else 'sentences'

for fold_num in range(1, n_splits+1):
    print('Сплит %s'%fold_num)
    torch.manual_seed(0)
    np.random.seed(0)
    best_model_path = './Model_weights/rubert_int_test_size_033_es_split_%s.pt'%fold_num
    
    #фразы могут быть с контекстом или без контекста, в зависимости от флага ADD_CONTEXT
    train_phrases, train_concepts = all_splits[fold_num]['train'][input_format], all_splits[fold_num]['train']['concepts']
    val_phrases, val_concepts = all_splits[fold_num]['validation'][input_format], all_splits[fold_num]['validation']['concepts']
    test_phrases, test_concepts = all_splits[fold_num]['test'][input_format], all_splits[fold_num]['test']['concepts']
    
    if not ADD_CONTEXT:
        ds_train = MedNormDataset(train_phrases, train_concepts, CV, use_cuda=USE_CUDA)
        ds_test = MedNormDataset(test_phrases, test_concepts, CV, use_cuda=USE_CUDA)
        ds_val = MedNormDataset(val_phrases, val_concepts, CV, use_cuda=USE_CUDA)
    else:
        ds_train = MedNormContextDataset(train_phrases, train_concepts, CV, use_cuda=USE_CUDA)
        ds_test = MedNormContextDataset(test_phrases, test_concepts, CV, use_cuda=USE_CUDA)
        ds_val = MedNormContextDataset(val_phrases, val_concepts, CV, use_cuda=USE_CUDA)
        
    #net = CADEC_SoTa(transformer_model_path, CV.thesaurus_embeddings)
    #перемещение модели на вычислительные мощности, либо cpu, либо cuda
    net = torch.load('./Model_weights/rubert_only_additional_corp_v2_30ep.pt')
    device = 'cuda' if USE_CUDA else 'cpu'
    net.to(device)
    
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.AdamW(net.parameters(), lr=0.0002) #, cycle_momentum=True
    if USE_CYCLR:
        scheduler = CyclicLR(optimizer, base_lr=0.00002, max_lr=0.0002, cycle_momentum=False)
    scaler = torch.cuda.amp.GradScaler()
    early_stopping = EarlyStopping(path=best_model_path, mode='max', verbose=True)

    trainloader = torch.utils.data.DataLoader(ds_train, batch_size=batch_size,
                                              shuffle=False, num_workers=0)
    testloader = torch.utils.data.DataLoader(ds_test, batch_size=1, shuffle=False)
    valloader = torch.utils.data.DataLoader(ds_val, batch_size=1, shuffle=False)

    valid_evaluator = Evaluator(train_concepts, val_concepts)
    test_evaluator = Evaluator(train_concepts, test_concepts)

    initial_loss = None

    for epoch in range(1, epochs+1):
        #обучение модели в эпохе
        net.train()
        with tqdm(trainloader, unit="batch") as tepoch:
            for data in tepoch:

                tepoch.set_description(f"Epoch {epoch}")

                inputs = data['tokenized_phrases']
                labels = data['one_hot_labels']

                optimizer.zero_grad()
                if USE_CYCLR:
                    with torch.cuda.amp.autocast():
                        outputs = net(inputs)['output']
                        loss = criterion(outputs, labels)
                    scaler.scale(loss).backward()
                    scaler.step(optimizer)
                    if CYCLIC_LR:
                        scheduler.step()
                    scaler.update()
                else:
                    outputs = net(inputs)['output']
                    loss = criterion(outputs, labels)
                    loss.backward()
                    optimizer.step()
                    if USE_CYCLR:
                        scheduler.step()
                if initial_loss is None:
                    initial_loss = loss.item()
                tepoch.set_postfix(loss_decrease = str(initial_loss/loss.item()))
        #расчет точностей модели после эпохи
        net.eval()
        model_answers=[]
        with tqdm(valloader, unit="batch") as eval_process:
            for data in eval_process:

                inputs = data['tokenized_phrases']

                with torch.no_grad():
                    outputs_dict = net(inputs)
                    pred_meddra_code = CV.meddra_codes[outputs_dict['output'].argmax()]

                model_answers.append(pred_meddra_code)

        valid_evaluator.get_all_f1_scores(model_answers)
        early_stopping(valid_evaluator.f1_scores_df['All']['f1 macro']['score'], net)
        if early_stopping.early_stop:
            print("Early stopping")
            break
    net.load_state_dict(torch.load(best_model_path))



    print('Evaluating on test dataset')
    net.eval()
    model_answers=[]
    with tqdm(testloader, unit="batch") as eval_process:
        for data in eval_process:

            inputs = data['tokenized_phrases']

            with torch.no_grad():
                outputs_dict = net(inputs)
                pred_meddra_code = CV.meddra_codes[outputs_dict['output'].argmax()]

            model_answers.append(pred_meddra_code)

        test_evaluator.get_all_f1_scores(model_answers)
    all_splits_score_dfs.append(test_evaluator.f1_scores_df)

columns_tuples = [('All', 'f1 micro'), 
                 ('All', 'f1 macro'),

                 ('In VOC', 'f1 micro'), 
                 ('In VOC', 'f1 macro'),

                 ('Out of VOC', 'f1 micro'), 
                 ('Out of VOC', 'f1 macro')]
col_index = pd.MultiIndex.from_tuples(columns_tuples)
f1_mean_scores_df = pd.DataFrame(columns=col_index, index=['mean score', 'std'])
f1_mean_scores_df.loc['std', :] = np.std(np.array(all_splits_score_dfs), axis = 0)
f1_mean_scores_df.loc['mean score', :] = np.mean(np.array(all_splits_score_dfs), axis = 0)
f1_mean_scores_df

Сплит 1


100%|██████████| 630/630 [00:07<00:00, 88.19batch/s]


All              In VOC           Out of VOC         
       f1 micro  f1 macro  f1 micro  f1 macro   f1 micro f1 macro
score  0.777778  0.481684  0.782748  0.499963        0.0      0.0

Validation metric increased (-inf --> 0.481684).  Saving model ...


100%|██████████| 630/630 [00:06<00:00, 91.42batch/s]


...

All              In VOC           Out of VOC         
       f1 micro  f1 macro  f1 micro  f1 macro   f1 micro f1 macro
score  0.828571  0.573159  0.832536  0.587425        0.0      0.0

EarlyStopping counter: 7 out of 7
Early stopping
Evaluating on test dataset


100%|██████████| 1573/1573 [00:15<00:00, 101.67batch/s]


All             In VOC           Out of VOC         
       f1 micro f1 macro  f1 micro  f1 macro   f1 micro f1 macro
score  0.837254  0.49156  0.844772  0.534695        0.0      0.0

All              In VOC           Out of VOC          
            f1 micro  f1 macro  f1 micro  f1 macro   f1 micro  f1 macro
mean score  0.846231  0.543905  0.851791  0.575738   0.071429  0.040908
std         0.006787  0.027927  0.006924  0.021902   0.063888  0.035263

<h2>SRoberta, Roberta на упоминаниях CADEC, которые содержат общие с корпусом RDRS PT коды</h2>

In [ ]:
df = pd.read_csv('../../Data/CADEC_intersection_part.csv')
df = df[df['pt_id']!='CONCEPT_LESS']
df['pt_id'] =  df['meddra'].apply(lambda x: str(x).split('|')[0])
mask = df['pt_id'].apply(lambda x: True if x in CV.meddra_codes else False)
df = df[mask]

folds = []
#df['pt_id'] = df['pt_id'].apply(lambda x: str(x))
concepts = df['pt_id'].to_list()
terms = df['phrase'].to_list()

for train_index, test_index in kf.split(df['phrase']):
    train_phrases = list(terms[train_index])
    test_phrases = list(terms[test_index])
    train_concepts = list(concepts[train_index])
    test_concepts = list(concepts[test_index])
    folds.append([[train_phrases, train_concepts], [test_phrases, test_concepts]])

In [ ]:
for fold_num in range(0, n_splits):
    print('exp: %s\n'%fold_num)
    train_phrases, train_concepts = list(folds[fold_num][0][0]), list(folds[fold_num][0][1])
    test_phrases, test_concepts = list(folds[fold_num][1][0]), list(folds[fold_num][1][1])

    ds_train = MedNormDataset(train_phrases, train_concepts, CV, use_cuda=True)
    ds_test = MedNormDataset(test_phrases, test_concepts, CV, use_cuda=True)

    torch.manual_seed(0)
    np.random.seed(0)
    
    net = CADEC_SoTa(transformer_model_path, torch.tensor(CV.thesaurus_embeddings))
    #перемещение модели на вычислительные мощности, либо cpu, либо cuda
    device = 'cuda' if USE_CUDA else 'cpu'
    net.to(device)
    print('Net loaded')
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.AdamW(net.parameters(), lr=0.0001) #, cycle_momentum=True

    scaler = torch.cuda.amp.GradScaler()
    evaluator = Evaluator(train_concepts, test_concepts)
    
    trainloader = torch.utils.data.DataLoader(ds_train, batch_size=batch_size,
                                          shuffle=False, num_workers=0)
    testloader = torch.utils.data.DataLoader(ds_test, batch_size=1, shuffle=False)


    initial_loss = None

    for epoch in range(1, epochs+1):
        #обучение модели в эпохе
        net.train()
        with tqdm(trainloader, unit="batch") as tepoch:
            for data in tepoch:

                tepoch.set_description(f"Epoch {epoch}")

                inputs = data['tokenized_phrases']
                labels = data['one_hot_labels']

                optimizer.zero_grad()
                if USE_CUDA=='cuda':
                    with torch.cuda.amp.autocast():
                        outputs = net(inputs)['output']
                        loss = criterion(outputs, labels)
                    scaler.scale(loss).backward()
                    scaler.step(optimizer)
                    scaler.update()
                else:
                    outputs = net(inputs)['output']
                    loss = criterion(outputs, labels)
                    loss.backward()
                    optimizer.step()
                if initial_loss is None:
                    initial_loss = loss.item()
                tepoch.set_postfix(loss_decrease = str(initial_loss/loss.item()))
    #расчет точностей модели после эпохи
    net.eval()
    model_answers=[]
    real_answers=[]
    with tqdm(testloader, unit="batch") as eval_process:
        for data in eval_process:

            inputs = data['tokenized_phrases']

            with torch.no_grad():
                outputs_dict = net(inputs)
                pred_meddra_code = CV.meddra_codes[outputs_dict['output'].argmax()]


            model_answers.append(pred_meddra_code)
            real_answers.append(data['label_codes'][0])
    evaluator.get_all_f1_scores(model_answers)

In [20]:
all_res = [[0.745816,  0.282695,  0.787387,  0.380494,   0.008475,  0.005435],
 [0.760033,  0.30586,  0.801036,  0.418271,        0.0,      0], 
 [0.751041,  0.302117,  0.795414,  0.404055,        0.0,      0.0], 
 [0.729866,  0.251004,  0.77186,  0.342622,   0.007634,  0.00319], 
 [0.713244,  0.267209,  0.755146,  0.374987,        0.0,     0.0]]

In [22]:
import numpy as np
import pandas as pd

columns_tuples = [('All', 'f1 micro'), 
                 ('All', 'f1 macro'),

                 ('In VOC', 'f1 micro'), 
                 ('In VOC', 'f1 macro'),

                 ('Out of VOC', 'f1 micro'), 
                 ('Out of VOC', 'f1 macro')]
col_index = pd.MultiIndex.from_tuples(columns_tuples)
f1_mean_scores_df = pd.DataFrame(columns=col_index, index=['mean score', 'std'])
f1_mean_scores_df.loc['std', :] = np.std(np.array(all_res), axis = 0)
f1_mean_scores_df.loc['mean score', :] = np.mean(np.array(all_res), axis = 0)
f1_mean_scores_df

All              In VOC           Out of VOC          
            f1 micro  f1 macro  f1 micro  f1 macro   f1 micro  f1 macro
mean score      0.74  0.281777  0.782169  0.384086   0.003222  0.001725
std         0.016587  0.020755  0.016704  0.026009   0.003955  0.002229